# Lecture 18. Working with Views (Hands On)

## How to

In this notebook, we will create and explore views.

We are going to first create one of each type of views.

Then in another notebook, we will explore the differences between how each one behaves in a new session.

## Preparing Sample Data

We start by creating a table of data to be used in this demonstration.

Here, we are creating a table called smartphones.
That has ID, name, and the brand of the phone, and the release year.

And we are inserting some data in this table.

In [0]:
%sql
USE CATALOG hive_metastore;

CREATE TABLE IF NOT EXISTS smartphones
(id INT, name STRING, brand STRING, year INT);

INSERT INTO smartphones
VALUES (1, 'iPhone 14', 'Apple', 2022),
      (2, 'iPhone 13', 'Apple', 2021),
      (3, 'iPhone 6', 'Apple', 2014),
      (4, 'iPad Air', 'Apple', 2013),
      (5, 'Galaxy S22', 'Samsung', 2022),
      (6, 'Galaxy Z Fold', 'Samsung', 2022),
      (7, 'Galaxy S9', 'Samsung', 2016),
      (8, '12 Pro', 'Xiaomi', 2022),
      (9, 'Redmi 11T Pro', 'Xiaomi', 2022),
      (10, 'Redmi Note 11', 'Xiaomi', 2021)

num_affected_rows,num_inserted_rows
10,10


The 10 lines have been inserted.

We can check the created table in the Catalog Explorer.

<div style="text-align: center;">
<img src="../../assets/images/Screen-Captures/Views Preparing Sample Data.jpg" style="width:1280px" >
</div> 

Also, we can use the `SHOW TABLES` Command to show the list of tables and views as well.

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
default,smartphones,false


As we can see here, we have one table called `smartphones` in the default database.

## Creating Stored Views

Let us now create a stored view that shows only Apple's phones.

Here we use the Create View statement followed by the view name.

In our case, it called `view_apple_phones`, and with the `AS` keyword, we specify our logical query.

Here we are selecting from smartphone table all records having a brand equal Apple.

In [0]:
%sql
CREATE VIEW view_apple_phones
AS  SELECT * 
    FROM smartphones 
    WHERE brand = 'Apple';

The view has been successfully created.

Let us now query our view.

To do so, we use the standard `SELECT` statement just as if it were a table.

In [0]:
%sql
SELECT * FROM view_apple_phones;

id,name,brand,year
1,iPhone 14,Apple,2022
2,iPhone 13,Apple,2021
3,iPhone 6,Apple,2014
4,iPad Air,Apple,2013


Here are all the apple phones retrieved from the table `smartphones`.

And remember, each time a view is queried, the underlying logical query of the view is executed against the table.

Let us run `SHOW TABLES` command again.

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
default,smartphones,false
default,view_apple_phones,false


We see that the view has been persisted to the default database.

And of course it is not a temporary object.

We can confirm this also from the Catalog Explorer.
Here is our view.

<div style="text-align: center;">
<img src="../../assets/images/Screen-Captures/view_apple_phones.jpg" style="width:1280px" >
</div> 

## Creating Temporary Views

Let us now create a temporary view.
The syntax is very similar, but we add a `TEMPORARY` keyword or simply a `TEMP` keyword.

The logical query of this view is simply retrieving the unique list of the brands in our smartphones table.

In [0]:
%sql
CREATE TEMP VIEW temp_view_phones_brands
AS  SELECT DISTINCT brand
    FROM smartphones;

SELECT * FROM temp_view_phones_brands;

brand
Xiaomi
Samsung
Apple


Here is the list of brands retrieved by the temporary review.

Let us run the `SHOW TABLES` command again.

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
default,smartphones,false
default,view_apple_phones,false
,temp_view_phones_brands,true


Here, we see that the temporary view has been added to the list.

The `isTemporary` column shows that this view is indeed a temporary object.

And since it is temporary, it is not persisted to any database.

## Creating Global Temporary Views

Lastly, we create a global temporary view.
Here, we simply add the `GLOBAL` keyword to the command.

Our global temporary view is called `global_temp_view_latest_phones`,
since the logical query here is retrieving all the smartphones from our table that has a releasing year newer than 2020.

And we order the result in descending order
to show first the most recent phones.

In [0]:
%sql
CREATE GLOBAL TEMP VIEW global_temp_view_latest_phones
AS SELECT * FROM smartphones
    WHERE year > 2020
    ORDER BY year DESC;

To query a global temporary view in a `SELECT` statement,
we need to use the `global_temp` database qualifier, 
which is in fact a *temporary database* in the cluster.

In [0]:
%sql
SELECT * FROM global_temp.global_temp_view_latest_phones;

id,name,brand,year
1,iPhone 14,Apple,2022
5,Galaxy S22,Samsung,2022
6,Galaxy Z Fold,Samsung,2022
8,12 Pro,Xiaomi,2022
9,Redmi 11T Pro,Xiaomi,2022
2,iPhone 13,Apple,2021
10,Redmi Note 11,Xiaomi,2021


Before we move on, let us review one last time the database table and views. 

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
default,smartphones,false
default,view_apple_phones,false
,temp_view_phones_brands,true


Our global temporary view is not listed here 
as it is tied to the `global_temp` database.

To show tables and views in the `global_temp` database, 
we use the command `SHOW TABLES IN` and we specify the database, 
in our case, it's `global_temp`.

In [0]:
%sql
SHOW TABLES IN global_temp;

database,tableName,isTemporary
global_temp,global_temp_view_latest_phones,true
,temp_view_phones_brands,true


Here, we can see the `global_temp_view_latest_phones`, which is tied to the `global_temp` database, which is, of course, a temporary object.

And since our `temp_view_phones_brands` is not tied to any database, it is usually shown with every `SHOW TABLES` command.

Next, we are going to demonstrate how tables and views are persisted across multiple sessions, and how temporary views are not.

To do so, we are going to create a [new notebook](./Lecture-18-working-with-views-hands-on-part-2.ipynb), which means a new Spark session, and explore how our created views will behave there.

In [0]:
%sql
SHOW TABLES;

database,tableName,isTemporary
default,smartphones,false
default,view_apple_phones,false
,temp_view_phones_brands,true
